In [1]:
%matplotlib widget
%load_ext autoreload

%autoreload 2

import numpy as np

In [2]:
from helper.laax_gps import get_all_laax_slopes, get_geopoints_from_slopes

slopes = get_all_laax_slopes(force=False)
all_geo_points = get_geopoints_from_slopes(slopes)

In [3]:
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource

output_notebook()
from helper.plots import plot_ski_slopes

show(plot_ski_slopes(slopes))

Loading BokehJS ...

In [4]:
from helper.laax_gps import get_gps_track
from helper.plots import plot_gps_track, plot_gps_altitude_distance, COLOR_MAP
track = get_gps_track('./data/Laax_fri_Nachmittag.gpx')
slope_map = plot_ski_slopes(slopes, color_map={'intermediate': 'lightblue', 'easy': 'lightblue', 'freeride': 'lightblue', 'advanced': 'lightblue'}, show_altitude=False)
gps_source = ColumnDataSource(track)

p1 = plot_gps_track(gps_source, color='tomato', overlay_onto=slope_map)
p2 = plot_gps_altitude_distance(gps_source, color='tomato')
show(gridplot([[p1, p2]]))

In [5]:
from helper.height_segmentation import find_rides_arne

rides = find_rides_arne(track['el'])
starts = np.where(rides.astype(int).diff() == 1)[0]
ends = np.where(rides.astype(int).diff() == -1)[0]

In [6]:
from bokeh.models import BoxAnnotation

p = plot_gps_altitude_distance(gps_source, color='tomato')

for s, e in zip(starts, ends):
    a = BoxAnnotation(left=track.iloc[s].t, right=track.iloc[e].t, fill_alpha=0.2, fill_color='blue')
    p.add_layout(a)
show(p)

In [12]:
from helper.laax_gps import find_closest_gps_track

ride_track = track[rides].copy()

ride_track['slope'] = find_closest_gps_track(ride_track, all_geo_points)

/home/arne/Documents/repos/work/laax/helper/laax_gps.py:100: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return closest.rolling(60).apply(lambda x: mode(x)[0]).fillna(method='backfill')


In [15]:
from helper.laax_gps import find_closest_gps_track

ride_track = track[find_rides_arne(track['el'])].copy()

ride_track['slope'] = find_closest_gps_track(ride_track, all_geo_points)
ride_track['difficulty'] = ride_track['slope'].dropna().apply(lambda x: slopes[int(x)]['difficulty'])
ride_track['difficulty_c'] = ride_track['difficulty'].apply(lambda x: COLOR_MAP.get(x, 'lightblue'))
ride_track_source = ColumnDataSource(ride_track)

/home/arne/Documents/repos/work/laax/helper/laax_gps.py:100: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return closest.rolling(60).apply(lambda x: mode(x)[0]).fillna(method='backfill')


In [16]:
p1 = plot_gps_track(ride_track_source, color='difficulty_c', overlay_onto=slope_map)
show(p1)

# Get a Reward!

In [10]:
POINT_MAP = {
        'intermediate': 2,
        'easy': 1,
        'freeride': 3,
        'advanced': 3
    }

ride_track['points'] = ride_track['difficulty'].apply(lambda x: POINT_MAP.get(x, 0)/60)

points = ride_track['points'].sum().round()
points

37.0

In [11]:
#send_point_token(points)